In [1]:
import sys
from collections import defaultdict

from scipy.io import wavfile
import numpy as np
from numpy.lib.stride_tricks import as_strided
import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, Conv2D, BatchNormalization, Conv1D
from tensorflow.keras.layers import MaxPool2D, Dropout, Permute, Flatten, Dense, MaxPool1D
from tensorflow.keras.models import Model
from shutil import copyfile
# import pyhocon
import os
import matplotlib.pyplot as plt
import pandas as pd
# from pydub import AudioSegment
from resampy import resample
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import pickle
from tqdm import tqdm
import math
import h5py
import os
# import faiss
from sklearn.model_selection import LeaveOneOut
from scipy import stats
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pickle
from spacy import displacy
from scipy import interpolate
from scipy import signal 
from multiprocess import Pool

In [2]:
def get_pitch_data(pitch_file_path):
    data = pd.read_csv(pitch_file_path, sep='\t')
    return np.array([p for p in data.values[:,1] if p!=0])

In [5]:
def fix_paths(path, add_mp3=False):
    fixed_path = path.replace('&', '_')
    fixed_path = fixed_path.replace(':', '_')
    fixed_path = fixed_path.replace('\'', '_')

    return fixed_path

def copy_pitch_file(old_path, mbid, lm_file):
    current_path = os.path.dirname(os.path.abspath("__file__"))
    
    old_path = old_path.replace('audio', 'features')
    new_path = 'data\\pitches\\' + mbid +'.pitch'
    old_path = os.path.join(current_path, old_path)
    new_path = os.path.join(current_path, new_path)
    old_path = '\\\\?\\' + old_path.replace('/', '\\')
    new_path = '\\\\?\\' + new_path.replace('/', '\\')
    lm_file[mbid] = get_pitch_data(old_path)
    
    copyfile(old_path,new_path)
    return mbid
    
def fetch_tonic(data, data_path):
    paths = data['path']
    mbids = data['mbid']
    tonic_list = []
    tonic_fine_list = []
    wav_path_list = []
    audio_data_path = os.path.join(data_path, 'audio')
    audio_lens = []
    # mp3_file_path = []
    current_path = os.path.dirname(os.path.abspath("__file__"))
    for mbid, path in tqdm(zip(mbids, paths)):
        
        path = r'' + path
        path = fix_paths(path)
        feature_path = path.replace('/audio/', '/features/')
        tonic_path = os.path.join(feature_path).replace('/', '\\') + '.tonic'
        tonic_path = os.path.join(current_path, tonic_path)
        tonic_path = '\\\\?\\' + tonic_path
        
        tonic_fine_path = os.path.join(feature_path).replace('/', '\\') + '.tonicFine'
        tonic_fine_path = os.path.join(current_path, tonic_fine_path)
        tonic_fine_path = '\\\\?\\' + tonic_fine_path
        
        mp3_file = os.path.join(feature_path).replace('/', '\\') + '.mp3'
        mp3_file = os.path.join(current_path, mp3_file)
        mp3_file = '\\\\?\\' + mp3_file
        
        if os.path.exists(tonic_path):
            
            with open(tonic_path, 'r') as f:
                tonic = f.readline().strip()
                tonic_list.append(tonic)
        else:
            print(tonic_path)
            tonic_list.append(-1)

        if os.path.exists(tonic_fine_path):
            with open(tonic_fine_path, 'r') as f:
                tonic_fine = f.readline().strip()
                tonic_fine_list.append(tonic_fine)
        else:
            tonic_fine_list.append(tonic_list[-1])

    data['path'] = data['path'].map(lambda x: os.path.join(x+'.pitch'))
    data['tonic'] = tonic_list
    data['tonic_fine'] = tonic_fine_list

    return data

def create_data_file():
    
    folder_path = 'data/'
    traditions = ['Hindustani', 'Carnatic']
    lm_file = 'pitch_data.h5'
    with h5py.File('data/RagaDataset/pitch_data.h5', "w") as lm_file:
        for trad in traditions:

            data_path = 'data/RagaDataset/{}/data.tsv'.format(trad)
            path_mbid_ragaid = os.path.join(folder_path,'RagaDataset', trad, '_info_', 'path_mbid_ragaid.txt')
            df = pd.read_csv(path_mbid_ragaid, names=['path', 'mbid', 'rag_id'], sep='\t')
            df['path'] = df['path'].map(lambda x: os.path.join(folder_path, x))
            df = fetch_tonic(df, folder_path)
            grouped = df.groupby(['rag_id'])

            ragaId_to_ragaName_mapping = os.path.join(folder_path,'RagaDataset', trad, '_info_',
                                                      'ragaId_to_ragaName_mapping.txt')
            ragaId_to_ragaName = pd.read_csv(ragaId_to_ragaName_mapping, sep='\t', names=['rag_id', 'rag_name'])

            ragaId_to_ragaName['labels'] = np.arange(ragaId_to_ragaName.shape[0])
            ragaId_to_ragaName = ragaId_to_ragaName.set_index(['rag_id'])

            data_list = []
            lbl = 0
            for k, v in grouped:
                v['rag_name'] = v['rag_id'].map(lambda x: ragaId_to_ragaName.loc[x]['rag_name'])
                v['labels'] = [lbl] * v.shape[0]
                v['path'] = v['path'].map(lambda x: fix_paths(x, False))
                v = v.reset_index(drop=True)
                data_list.append(v)
                lbl += 1

            data_list = pd.concat(data_list)
            data_list['new_path'] = data_list.apply(lambda x: copy_pitch_file(x['path'], x['mbid'], lm_file), axis=1)
            data_list.to_csv(data_path, sep='\t', index=False)

In [12]:
# create_data_file()

In [27]:
def freq_to_cents_np(freq, cents_mapping, std=25):
    frequency_reference = 10
    c_true = 1200 * np.log2((np.array(freq)+1e-5) / frequency_reference)
    c_true = np.expand_dims(c_true, 1)
    cents_mapping = np.tile(np.expand_dims(cents_mapping,0), [c_true.shape[0],1])
    target = np.exp(-(cents_mapping - c_true) ** 2 / (2 * std ** 2))
    pitch_cent = np.sum(target.reshape([c_true.shape[0], 6, 120]), 1)
    return pitch_cent
    

In [44]:
def freq_to_cents(freq, cents_mapping, std=25):
    frequency_reference = 10
    c_true = 1200 * math.log((freq+1e-5) / frequency_reference,2)
    target = np.exp(-(cents_mapping - c_true) ** 2 / (2 * std ** 2))
    pitch_cent = np.sum(target.reshape([6, 120]), 0)
    return pitch_cent
    

In [28]:
def get_pitchvalues(pitches_arr):
    cents_mapping = np.linspace(0, 7190, 720) + 2051.1487628680297
    return freq_to_cents_np(pitches_arr, cents_mapping)

In [29]:
def reorder_tonic(pitchvalue_prob, tonic_freq, tonic_offset=0):
    cents_mapping = np.linspace(0, 7190, 720) + 2051.1487628680297
    tonic_pv_arr = freq_to_cents(tonic_freq, cents_mapping)
    tonic_pv = np.argmax(tonic_pv_arr) + tonic_offset
    return np.roll(pitchvalue_prob, -tonic_pv, axis=1)

In [30]:
def normalize(z):
    z_min = np.min(z)
    return (z - z_min)/(np.max(z)-z_min+1e-6)

In [31]:
def compare(a,b,x,asc):
    if not asc:
        a, b = b, a
    if a <= modulo_add(a,x) <= modulo_add(a,b):
        return True
    return False

In [32]:
def modulo(x):
    return x%120

In [33]:
def modulo_add(x,y):
    mx = modulo(x)
    my = modulo(y)
    if mx>my:
        return my+120
    return my

In [34]:
def get_dist_btw_idx(pitchvalue_prob, start_idx, end_idx):
    dist = 0
    for i in range(start_idx, end_idx+1):
        dist+=pitchvalue_prob[i]
    return dist

def get_dist_btw_shortlisted_idxs(pitchvalue_prob,shortlisted_idxs, off_start=0, off_end=None):
    if off_end is None:
        off_end = len(pitchvalue_prob)-1
    dist = 0
    for sidx in shortlisted_idxs:
        i1, i2, i3, i4 = sidx[0], sidx[1], sidx[2], sidx[3]
        if i2<off_start:
            continue
        if i3>off_end:
            continue
        if i1<=off_start<=i2:
            i1 = off_start
        if i3<=off_end<=i4:
            i4 = off_end
        
        dist += get_dist_btw_idx(pitchvalue_prob, i1, i4)
    return normalize(dist)


In [35]:
def update_shortlisted_index(shortlisted_index, pitch_st_mapping, start_index, end_index):
    psm_ss = pitch_st_mapping[start_index][0] 
    psm_se = pitch_st_mapping[start_index][1] 
    psm_es = pitch_st_mapping[end_index][0]
    psm_ee = pitch_st_mapping[end_index][1] 
    
    shortlisted_index.append((psm_ss, psm_se, psm_es, psm_ee))
    

In [36]:
def roundp(p):
    return int(round(p/10))

In [37]:
def compute_spd_ps_pe(pitchvalue_prob, start_idx, pitches_arg, pitch_st_mapping, ps, pe, asc, mbid, relax=4):
    n = len(pitches_arg)
    k=0
    start = True
    end = False
    b = 0
    start_id = 0
    end_id = 0
    si=0
    prev_s = None
    start_index = -1
    end_index = -1
    idx = start_idx[si]
    dist_pres = False
    dist_added = False
    shortlisted_index = []
    while idx<n:
        
        p = pitches_arg[idx]

        if start and end and p!=pe:
            update_shortlisted_index(shortlisted_index, pitch_st_mapping, start_index, end_index)
            start = False
            end = False
            start_index = -1
            end_index = -1
            dist_pres = True

        if start and p==pe:
            end = True  # This verifies Equation 14; Page 4
            end_index = idx

        if p==ps:
            start = True  # This verifies Equation 12; Page 4
            start_index = idx
            
        if not (compare(ps, pe, p, asc)):
#             prev_dist = 0
            start = False
            end = False
            start_index = -1
            end_index = -1
            
        if p==ps:
            si+=1
        if not start:
            if si>=len(start_idx):
                break
            else:
                idx = start_idx[si]
                idx-=1
        idx+=1
    
    # This handles an edge case where prev_dist is not empty but not yet been added to cum_pitch_dist
    if start and end:
        update_shortlisted_index(shortlisted_index, pitch_st_mapping, start_index, end_index)
    
    if not dist_pres:
        update_shortlisted_index(shortlisted_index, pitch_st_mapping, 0, n-1)
    return shortlisted_index


def compute_spd_ps_pe_parallel(spd_arg):
    pitchvalue_prob, start_idx, pitches_arg, pitch_st_mapping, ps, pe, asc, mbid = spd_arg[0], spd_arg[1], spd_arg[2], spd_arg[3], spd_arg[4], spd_arg[5], spd_arg[6], spd_arg[7]
    
    def modulo(x):
        return x%120

    def modulo_add(x,y):
        mx = modulo(x)
        my = modulo(y)
        if mx>my:
            return my+120
        return my
        
    def compare(a,b,x,asc):
        if not asc:
            a, b = b, a
        if a <= modulo_add(a,x) <= modulo_add(a,b):
            return True
        return False
    
    def update_shortlisted_index(shortlisted_index, pitch_st_mapping, start_index, end_index):
        psm_ss = pitch_st_mapping[start_index][0] 
        psm_se = pitch_st_mapping[start_index][1] 
        psm_es = pitch_st_mapping[end_index][0]
        psm_ee = pitch_st_mapping[end_index][1] 

        shortlisted_index.append((psm_ss, psm_se, psm_es, psm_ee))
    
    relax = 4
    n = len(pitches_arg)
    k=0
    start = True
    end = False
    b = 0
    start_id = 0
    end_id = 0
    si=0
    prev_s = None
    start_index = -1
    end_index = -1
    idx = start_idx[si]
    dist_pres = False
    dist_added = False
    shortlisted_index = []
    while idx<n:
        
        p = pitches_arg[idx]

        if start and end and p!=pe:
            update_shortlisted_index(shortlisted_index, pitch_st_mapping, start_index, end_index)
            start = False
            end = False
            start_index = -1
            end_index = -1
            dist_pres = True

        if start and p==pe:
            end = True  # This verifies Equation 14; Page 4
            end_index = idx
        if p==ps:
            start = True  # This verifies Equation 12; Page 4
            start_index = idx
            
        if not (compare(ps, pe, p, asc)):
#             prev_dist = 0
            start = False
            end = False
            start_index = -1
            end_index = -1
            
        if p==ps:
            si+=1
        if not start:
            if si>=len(start_idx):
                break
            else:
                idx = start_idx[si]
                idx-=1
        idx+=1
    
    # This handles an edge case where prev_dist is not empty but not yet been added to cum_pitch_dist
    if start and end:
        update_shortlisted_index(shortlisted_index, pitch_st_mapping, start_index, end_index)
    
    if not dist_pres:
        update_shortlisted_index(shortlisted_index, pitch_st_mapping, 0, n-1)
    return shortlisted_index


In [38]:
def get_all_smooth_pitch_values(std=25):
    c_note = freq_to_cents(32.7 * 2, std)
    all_notes = np.zeros([120, 120])
    for p in range(120):
        all_notes[p] = get_smooth_pitch_value(c_note, p)

    return all_notes, c_note

def get_smooth_pitch_value(c_note, note):
    return np.roll(c_note, note, axis=-1)

def gauss_smooth(raga_feat):
    all_notes, c_note = get_all_smooth_pitch_values(std=25)
    smooth = np.zeros([12,12,120,2])
    for i in range(12):
        for j in range(12):
            if i==j:
                continue
            for k in range(0,2):
                smooth[i,j,:,k] = gauss_smooth_util(raga_feat[i,j,:,k], all_notes)
    return smooth
                
def gauss_smooth_util(arr1, all_notes):
    smooth = 0
    for i in range(120):
        smooth = smooth + all_notes[i]*arr1[i]
    
    smooth = normalize(smooth)
    return smooth

In [39]:

def full_spd(pitchvalue_prob, pitches_arg, mbid, lm_file):
    pitch_dict, std_pitches, pitch_st_mapping = get_std_idx(pitches_arg)
    for asc in [True, False]:
        for s in range(0, 12, 1):
            start_idx = pitch_dict[s]
            for e in range(0, 12, 1):
                if s==e:
                    continue
                shortlisted_index = compute_spd_ps_pe(pitchvalue_prob, start_idx, std_pitches, pitch_st_mapping, s, e, asc, mbid)  
                lm_file["{}_{}_{}_{}".format(mbid, s, e, asc)] = shortlisted_index

In [40]:
def full_spd_parallel(pitchvalue_prob, pitches_arg, mbid, lm_file):
    pitch_dict, std_pitches, pitch_st_mapping = get_std_idx(pitches_arg)
    for asc in [True, False]:
        for s in range(0, 12, 1):
            start_idx = pitch_dict[s]
            par_inp = []
            end_vals = [e for e in range(0, 12, 1) if s!=e]
            for e in range(0, 12, 1):
                if s==e:
                    continue
                par_inp.append([pitchvalue_prob, start_idx, std_pitches, pitch_st_mapping, s, e, asc, mbid])
            with Pool(5) as p:
                shortlisted_indexes = p.map(compute_spd_ps_pe_parallel, par_inp)
            for ed, si in zip(end_vals, shortlisted_indexes):
                lm_file["{}_{}_{}_{}".format(mbid, s, ed, asc)] = si

In [41]:
def get_std_idx(pitches_arg, relax=4):
    pitch_dict = defaultdict(list)
    std_pitches = []
    pitch_st_mapping = []
    prev_p = None
    k=0
    for i, p in enumerate(pitches_arg):
        if prev_p is None:
            std_pitches.append(roundp(p))
            pitch_dict[roundp(p)].append(k)
        elif roundp(prev_p) != roundp(p):
            k+=1
            std_pitches.append(roundp(p))
            pitch_dict[roundp(p)].append(k)
        if k>=len(pitch_st_mapping):
            pitch_st_mapping.append([i,i])
        else:
            if pitch_st_mapping[-1][1]+1==i:
                pitch_st_mapping[-1][1] = i
            else:
                pitch_st_mapping.append([i,i])
        prev_p = p
        
    return pitch_dict, std_pitches, pitch_st_mapping

In [46]:
# Create an empty cache folder in RagaDataset/Hindustani and RagaDataset/Carnatic folders

def generate_spd_idx_all_files(tradition):
    pitch_dict = None
    data_file_path = 'data/RagaDataset/{}/data.tsv'.format(tradition)
    df = pd.read_csv(data_file_path, sep='\t')
    
    with h5py.File('data/RagaDataset/'+tradition+'/cache/spd_cache.h5', "w") as spd_idx_lm_file:
        with h5py.File('data/RagaDataset/pitch_data.h5', "r") as pitch_lm_file: 
            for row in df.iterrows():
                file_name = row[1]['new_path']
                tonic = row[1]['tonic_fine']
                mbid = row[1]['mbid']
                print(file_name)
                pitches = pitch_lm_file[mbid]
                pitchvalue_prob = get_pitchvalues(pitches)
                pitchvalue_prob = reorder_tonic(pitchvalue_prob, tonic)

                pitches_arg = np.argmax(pitchvalue_prob, axis=1)
                full_spd(pitchvalue_prob, pitches_arg, mbid, spd_idx_lm_file)


In [ ]:
generate_spd_idx_all_files('Hindustani')
# generate_spd_idx_all_files('Carnatic')


a7f6551d-9696-4713-becb-b22acf71a7db
d0781259-2c63-4b35-8ca3-75c19a98e275
99dcaebe-ab49-4fa3-ab6c-a9458143af8e
86d7779a-1610-4514-83a6-883997250abe
c02f714f-e4e6-4da6-af07-cea8a03c0e5c


In [49]:
def get_cliped_dist(s,e,asc,dist,clip=15):
    s10 = s*10
    e10 = e*10
    if asc:
        relax = clip
    else:
        relax = -clip
    i = modulo(s10-relax)
    j = modulo(e10+relax)
    m = 0
    while i!=j:
        if asc:
            i = modulo(i+1)
        else:
            i = modulo(i-1)
        m+=1
    dist_sliced = np.zeros(m)
    i = modulo(s10-relax)
    j = modulo(e10+relax)
    if (m<=abs(relax)):
        dist_sliced = dist
    else:
        m=0
        while i!=j:
            if asc:
                i = modulo(i+1)
            else:
                i = modulo(i-1)
            dist_sliced[m] = dist[i]
            m+=1
    return dist_sliced

def get_spd_from_idx(mbid, tonic, tradition, just_spd_lm_file, just_hist_lm_file, off_start=0, off_end=None):
    with h5py.File('data/RagaDataset/pitch_data.h5', "r") as pitch_lm_file:
        pitches = pitch_lm_file[mbid]
        pitchvalue_prob = get_pitchvalues(pitches)
        pitchvalue_prob = reorder_tonic(pitchvalue_prob, tonic)
        dist_hist = get_dist_btw_idx(pitchvalue_prob, 0, len(pitchvalue_prob)-1)
        dist_hist = normalize(dist_hist)
        with h5py.File('data/RagaDataset/'+tradition+'/cache/spd_cache.h5', "r") as spd_idx_lm_file:
            full_spd_dist = np.zeros([12,12,120,2])
            for asc in [True, False]:
                asc_int = 1-int(asc)
                for s in range(0,12,1):
                    for e in range(0, 12, 1):
                        if s==e:
                            full_spd_dist[s,e,:,asc_int] = dist_hist
                            continue
                        shortlisted_idxs = spd_idx_lm_file['{}_{}_{}_{}'.format(mbid, s, e, asc)]
                        dist = get_dist_btw_shortlisted_idxs(pitchvalue_prob,shortlisted_idxs, off_start, off_end)
                        full_spd_dist[s,e,:,asc_int] = dist
            just_spd_lm_file[mbid] = full_spd_dist
            just_hist_lm_file[mbid] = dist_hist
            

            

In [56]:
def generate_full_spd_cache(tradition):
    with h5py.File('data/RagaDataset/'+tradition+'/cache/just_spd_cache.h5', "w") as just_spd_lm_file:
        with h5py.File('data/RagaDataset/'+tradition+'/cache/just_hist_cache.h5', "w") as just_hist_lm_file:
            data_file_path = 'data/RagaDataset/{}/data.tsv'.format(tradition)
            df = pd.read_csv(data_file_path, sep='\t')
            for row in df.iterrows():
                mbid = row[1]['mbid']
                tonic = row[1]['tonic_fine']
                print(mbid)
                get_spd_from_idx(mbid, tonic, tradition, just_spd_lm_file, just_hist_lm_file)


In [54]:
# generate_full_spd_cache('Carnatic')
# generate_full_spd_cache('Hindustani')

In [57]:
class SPDKNN:
    def __init__(self,k=5):
        self.y = None
        self.knn = KNeighborsClassifier(n_neighbors=k, algorithm='ball_tree', metric=self.bhatta)
    
    def bhatta(self, hist1,  hist2):
        # calculate mean of hist1
        h1_ = np.mean(hist1)
        h2_ = np.mean(hist2)
        # calculate mean of hist2
    
        
        # calculate score
        score = np.sum(np.sqrt(np.multiply(hist1, hist2)))
        # print h1_,h2_,score;
        score = math.sqrt( 1 - ( 1 / math.sqrt(h1_*h2_*len(hist1)*len(hist2)) ) * score );
        return score

    def fit(self, X, y):
        self.knn.fit(X,y)
    
    def predict(self, X):
        return self.knn.predict_proba(X)

In [61]:
def train_model(tradition, only_save=False):
    data_file_path = 'data/RagaDataset/{}/data.tsv'.format(tradition)
    if tradition=='Hindustani':
        n_rows = 300
        n_labels = 30
    else:
        n_rows = 480
        n_labels = 40
    df = pd.read_csv(data_file_path, sep='\t')
    file_pred = 0
    weights = get_final_weights(tradition)
    with h5py.File('data/RagaDataset/'+tradition+'/cache/just_spd_cache.h5', "r") as just_spd_lm_file:
        with h5py.File('data/RagaDataset/'+tradition+'/cache/just_hist_cache.h5', "r") as just_hist_lm_file:
            with h5py.File('data/RagaDataset/'+tradition+'/cache/output_cache_clipped.h5', "w") as output_lm_file:
                for wd in range(0,250,10):
                    spd_knn = SPDKNN(k=5)
                    y_labels = df['labels'].values
                    if wd == 0:
                        feat = np.zeros([n_rows, 120])
                        for row in df.iterrows():
                            feat[row[0]] = just_hist_lm_file[row[1]['mbid']]
                    elif 0<wd<120:
#                         feat = np.zeros([300, 120*12*2])
                        feat = np.zeros([n_rows, 120*12*2])
                        feat = []
                        for row in df.iterrows():
                            mbid = row[1]['mbid']
                            feat_curr = None
                            feat_curr = []
                            for s in range(0,120,10):
                                e = modulo(s+wd)
                                if s==e:
                                    continue
                                s10 = roundp(s)
                                e10 = roundp(e)
                                hist_1 = just_spd_lm_file[mbid][s10,e10,:,0]
                                hist_2 = just_spd_lm_file[mbid][e10,s10,:,1]
                                hist_1 = get_cliped_dist(s,e,True,hist_1,clip=15)
                                hist_2 = get_cliped_dist(e,s,False,hist_2,clip=15)
                                feat_curr.append(hist_1)
                                feat_curr.append(hist_2)
                            feat_curr = np.concatenate(feat_curr, axis=-1)
                            feat.append(feat_curr)
                    elif 120<=wd<240:
                        feat = []
                        s = wd-120
                        for row in df.iterrows():
                            mbid = row[1]['mbid']
                            feat_curr = None
                            feat_curr = []
                            for e in range(0,120,10):
                                if s==e:
                                    continue
                                s10 = roundp(s)
                                e10 = roundp(e)
                                hist_1 = just_spd_lm_file[mbid][s10,e10,:,0]
                                hist_2 = just_spd_lm_file[mbid][e10,s10,:,1]
                                hist_1 = get_cliped_dist(s,e,True,hist_1,clip=15)
                                hist_2 = get_cliped_dist(e,s,False,hist_2,clip=15)
                                feat_curr.append(hist_1)
                                feat_curr.append(hist_2)
                            feat_curr = np.concatenate(feat_curr, axis=-1)
                            feat.append(feat_curr)
                    else:
                        feat = []
                        for row in df.iterrows():
                            mbid = row[1]['mbid']
                            hist_1 = np.array(just_spd_lm_file[mbid])
                            feat.append(np.reshape(hist_1, [-1]))
                    feat = np.array(feat)
                    loo = LeaveOneOut()
                    y_pred = np.zeros([n_rows,n_labels])
                    print('wd',wd)
                    if only_save:
                        spd_knn.fit(feat, y_labels)
                        with open('model/{}/spd_knn_{}.pkl'.format(tradition, wd), 'wb') as pkl_f:
                            pickle.dump(spd_knn, pkl_f)
                    else:
                        for train_index, test_index in loo.split(feat):  
                            spd_knn.fit(feat[train_index], y_labels[train_index])
                            y_pred_proba = spd_knn.predict(feat[test_index])
                            y_pred[test_index[0]] = y_pred_proba[0]
                        output_lm_file[str(wd)] = y_pred
                        file_pred += weights[int(wd/10)]*y_pred
                file_pred = np.argmax(file_pred, axis=1)
                pred_df = pd.DataFrame()
                pred_df['pred'] = file_pred
                pred_df['labels'] = y_labels
                pred_df.to_csv(f'Test_Pred_{t}.csv', index=False)
                accuracy = np.mean(file_pred==y_labels)*100
                print(f'{tradition}_{t}_{accuracy}')


In [100]:
# train_model('Carnatic', False)
train_model('Hindustani', False)

wd 0
wd 10
wd 20
wd 30
wd 40
wd 50
wd 60
wd 70
wd 80
wd 90
wd 100
wd 110
wd 120
wd 130
wd 140
wd 150
wd 160
wd 170
wd 180
wd 190
wd 200
wd 210
wd 220
wd 230
wd 240
Hindustani_0.25_98.33333333333333


In [67]:
train_model('Carnatic')
# train_model('Hindustani')

wd 0
(1, 120)
(1, 120)
(1, 120)
(1, 120)
(1, 120)
(1, 120)
(1, 120)
(1, 120)
(1, 120)
(1, 120)
(1, 120)
(1, 120)
(1, 120)
(1, 120)
(1, 120)


KeyboardInterrupt: 

In [62]:
def get_data(tradition):
    data_file_path = 'data/RagaDataset/{}/data.tsv'.format(tradition)
    if tradition=='Hindustani':
        n_rows = 300
        n_labels = 30
    else:
        n_rows = 480
        n_labels = 40
        
    df = pd.read_csv(data_file_path, sep='\t')
    X = np.zeros([n_rows, n_labels, 25])
    y = df['labels']
    with h5py.File('data/RagaDataset/'+tradition+'/cache/output_cache_clipped.h5', "r") as output_lm_file:
        outp = 0
        for i in range(0,250,10):
            X[:,:, roundp(i)] = output_lm_file[str(i)]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    return X_train, y_train, X_test, y_test

In [64]:
class BestWeights:
    def __init__(self, tradition):
        if tradition=='Hindustani':
            self.n_labels = 30
        else:
            self.n_labels = 40
        self.tradition = tradition
        self.model = self.build_model()
    
    def build_model(self):
        model = tf.keras.Sequential([tf.keras.layers.Input(shape=(self.n_labels,25)), tf.keras.layers.Dense(1), tf.keras.layers.Lambda(lambda x: tf.squeeze(x,2))])
        model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                     metrics=['accuracy'])
        return model
    
    def train(self, X, y):
        
        mcp_save = tf.keras.callbacks.ModelCheckpoint('data/RagaDataset/{}/cache/mdl_wts.weights.h5'.format(self.tradition), save_best_only=True, monitor='accuracy', mode='max', save_weights_only=True)
        self.model.fit(X,y, epochs=1000, batch_size=10, callbacks=[mcp_save])
    
    def test(self, X):
        self.model.load_weights('data/RagaDataset/{}/cache/mdl_wts.weights.h5'.format(self.tradition), skip_mismatch=False)
        print(list([a[0] for a in self.model.layers[0].weights[0].numpy()]))
        return self.model.predict(X)

In [96]:
tf.keras.backend.clear_session()
X_train, y_train, X_test, y_test = get_data('Carnatic')
X_train, y_train, X_test, y_test = get_data('Hindustani')
bw = BestWeights(tradition='Hindustani')


Epoch 1/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7973 - loss: 1.0163
Epoch 2/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8295 - loss: 0.9747
Epoch 3/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8057 - loss: 1.1177
Epoch 4/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8187 - loss: 0.8731
Epoch 5/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7972 - loss: 1.1790
Epoch 6/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8318 - loss: 0.9387
Epoch 7/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8227 - loss: 0.9501
Epoch 8/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8180 - loss: 1.1685
Epoch 9/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8056 - loss: 1.1442
Epoch 10/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8094 - loss: 0.9169
Epoch 11/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8366 - loss: 1.0211
Epoch 12/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8455 - loss: 0.7461
Epoch 136/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8107 - loss: 0.9559
Epoch 137/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8433 - loss: 0.8241
Epoch 138/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8440 - loss: 0.8904
Epoch 139/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8624 - loss: 0.6477
Epoch 140/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8403 - loss: 0.9200
Epoch 141/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8604 - loss: 0.7788
Epoch 142/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8516 - loss: 0.8440
Epoch 143/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8595 - loss: 0.8675
Epoch 144/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8338 - loss: 0.6619
Epoch 145/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8487 - loss: 0.6317
Epoch 146/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3m

KeyboardInterrupt: 

In [ ]:


tf.keras.backend.clear_session()
X_train, y_train, X_test, y_test = get_data('Carnatic')
bw = BestWeights(tradition='Carnatic')
bw.train(X_train, y_train.values)


In [111]:
bw = BestWeights('Hindustani')
tf.keras.backend.clear_session()
tf.keras.config.enable_unsafe_deserialization()
X_train, y_train, X_test, y_test = get_data('Hindustani')
y_pred = bw.test(np.concatenate([X_train, X_test], axis=0))
np.mean(np.argmax(y_pred, 1)==np.concatenate([y_train.values, y_test.values], axis=0))

C:\Users\vnarasinh\anaconda3\envs\env_1\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


[0.29106927, -0.03946995, 0.02425879, 0.283751, 0.023861948, 0.31980556, -0.20827645, 0.17380531, -0.0649386, -0.1427266, -0.053185567, 0.23119006, 0.20106639, 0.118402176, -0.24709608, 0.14900881, 0.12632905, -0.13513866, -0.047171496, 0.08214283, 0.026735343, -0.031288646, -0.24385814, 0.04932891, 0.2976105]
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


0.9833333333333333

In [ ]:
bw = BestWeights('Carnatic')
tf.keras.backend.clear_session()
tf.keras.config.enable_unsafe_deserialization()
X_train, y_train, X_test, y_test = get_data('Carnatic')
y_pred = bw.test(np.concatenate([X_train, X_test], axis=0))
np.mean(np.argmax(y_pred, 1)==np.concatenate([y_train.values, y_test.values], axis=0))

In [65]:
def get_final_weights(tradition):
    if tradition=='Hindustani':
        return [0.29106927, -0.03946995, 0.02425879, 0.283751, 0.023861948, 0.31980556, 
                -0.20827645, 0.17380531, -0.0649386, -0.1427266, -0.053185567, 0.23119006,
                0.20106639, 0.118402176, -0.24709608, 0.14900881, 0.12632905, -0.13513866, 
                -0.047171496, 0.08214283, 0.026735343, -0.031288646, -0.24385814, 0.04932891, 0.2976105]
    return [0.31490463, 0.21623285, 0.101201855, -0.05553831, 0.12762547, 0.062958464, 0.051889133,
            0.10306381, -0.15440132, 0.20237908, -0.021573946, 0.07168238, -0.0014262914, 0.1189323,
            -0.028502718, 0.1141022, 0.07979874, -0.054066267, -0.19121367, 0.026354773, 0.0811235,
            0.109708786, -0.12738012, -0.009295256, -0.032566402]
    
    

In [66]:
def get_spd(pitches, tonic, only_hist=False):
    off_start=0
    off_end=None
    pitchvalue_prob = get_pitchvalues(pitches)
    pitchvalue_prob = reorder_tonic(pitchvalue_prob, tonic)
    dist_hist = get_dist_btw_idx(pitchvalue_prob, 0, len(pitchvalue_prob)-1)
    dist_hist = normalize(dist_hist)
    if only_hist:
        return dist_hist
    
    pitches_arg = np.argmax(pitchvalue_prob, axis=1)
    lm_file = {}
    mbid = 'tempo_mbid'
    full_spd(pitchvalue_prob, pitches_arg, mbid, lm_file)
#     full_spd_parallel(pitchvalue_prob, pitches_arg, mbid, lm_file)
    full_spd_dist = np.zeros([12,12,120,2])
    for asc in [True, False]:
        asc_int = 1-int(asc)
        for s in range(0,12,1):
            for e in range(0, 12, 1):
                if s==e:
                    full_spd_dist[s,e,:,asc_int] = dist_hist
                    continue
                shortlisted_idxs = lm_file['{}_{}_{}_{}'.format(mbid, s, e, asc)]
                dist = get_dist_btw_shortlisted_idxs(pitchvalue_prob,shortlisted_idxs, off_start, off_end)
                full_spd_dist[s,e,:,asc_int] = dist

    return full_spd_dist


def change_tempo(pitches_sample, t):
    sample_data_pitch_resam = signal.resample(pitches_sample, int((1/t)*len(pitches_sample)))
    return sample_data_pitch_resam

def train_tempo_model(tradition, t, only_save=False):
    data_file_path = 'data/RagaDataset/{}/data.tsv'.format(tradition)
    if tradition=='Hindustani':
        n_rows = 300
        n_labels = 30
    else:
        n_rows = 480
        n_labels = 40
    df = pd.read_csv(data_file_path, sep='\t')
    weights = get_final_weights(tradition)
    
    
    with h5py.File('data/RagaDataset/'+tradition+f'/cache/just_spd_cache_tempo_{t}.h5', "w") as spd_tempo: 
        with h5py.File('data/RagaDataset/'+tradition+f'/cache/just_hist_cache_tempo_{t}.h5', "w") as hist_tempo:
            with h5py.File('../data/RagaDataset/pitch_data.h5', "r") as pitch_lm_file: 
                for row in df.iterrows():
                    mbid = row[1]['mbid']
                    print(mbid)
                    tonic_fine = row[1]['tonic_fine']
                    test_pitches = np.array(pitch_lm_file[mbid])
                    test_pitches = change_tempo(test_pitches, t)
                    test_pitches = np.clip(test_pitches, 0, float('inf'))
                    temp = get_spd(test_pitches, tonic_fine, False)
                    hist_tempo[mbid] = get_spd(test_pitches, tonic_fine, True)
                    spd_tempo[mbid] = temp

    file_pred = 0
    with h5py.File('data/'+tradition+'_just_spd_cache', "r") as just_spd_lm_file:
        with h5py.File('data/'+tradition+'_just_hist_cache', "r") as just_hist_lm_file:
            with h5py.File('data/RagaDataset/'+tradition+f'/cache/just_spd_cache_tempo_{t}.h5', "r") as spd_tempo:
                with h5py.File('data/RagaDataset/'+tradition+f'/cache/just_hist_cache_tempo_{t}.h5', "r") as hist_tempo:
                    for wd in range(0,250,10):
                        spd_knn = SPDKNN(k=5)
                        y_labels = df['labels'].values
                        if wd == 0:
                            feat = np.zeros([n_rows, 120])
                            test_feat = np.zeros([n_rows, 120])
                            for row in df.iterrows():
                                test_feat[row[0]] = hist_tempo[row[1]['mbid']]
                                feat[row[0]] = just_hist_lm_file[row[1]['mbid']]
                        elif 0<wd<120:
                            feat = []
                            test_feat = []
                            for row in df.iterrows():
                                mbid = row[1]['mbid']
                                feat_curr = []
                                test_feat_curr = []
#                                 test_spd = np.array(spd_tempo[mbid])
                                for s in range(0,120,10):
                                    e = modulo(s+wd)
                                    if s==e:
                                        continue
                                    s10 = roundp(s)
                                    e10 = roundp(e)
                                    test_hist_1 = spd_tempo[mbid][s10,e10,:,0]
                                    test_hist_2 = spd_tempo[mbid][e10,s10,:,1]
                                    test_hist_1 = get_cliped_dist(s,e,True,test_hist_1,clip=15)
                                    test_hist_2 = get_cliped_dist(e,s,False,test_hist_2,clip=15)
                                    test_feat_curr.append(test_hist_1)
                                    test_feat_curr.append(test_hist_2)

                                    hist_1 = just_spd_lm_file[mbid][s10,e10,:,0]
                                    hist_2 = just_spd_lm_file[mbid][e10,s10,:,1]
                                    hist_1 = get_cliped_dist(s,e,True,hist_1,clip=15)
                                    hist_2 = get_cliped_dist(e,s,False,hist_2,clip=15)
                                    feat_curr.append(hist_1)
                                    feat_curr.append(hist_2)
                                feat_curr = np.concatenate(feat_curr, axis=-1)
                                test_feat_curr = np.concatenate(test_feat_curr, axis=-1)
                                feat.append(feat_curr)
                                test_feat.append(test_feat_curr)
                        elif 120<=wd<240:
                            feat = []
                            test_feat = []
                            s = wd-120
                            for row in df.iterrows():
                                mbid = row[1]['mbid']
                                feat_curr = None
                                feat_curr = []
                                test_feat_curr = []
                                for e in range(0,120,10):
                                    if s==e:
                                        continue
                                    s10 = roundp(s)
                                    e10 = roundp(e)

                                    test_hist_1 = spd_tempo[mbid][s10,e10,:,0]
                                    test_hist_2 = spd_tempo[mbid][e10,s10,:,1]
#                                     test_hist_1 = just_spd_lm_file[mbid][s10,e10,:,0]
#                                     test_hist_2 = just_spd_lm_file[mbid][e10,s10,:,1]
                                    test_hist_1 = get_cliped_dist(s,e,True,test_hist_1,clip=15)
                                    test_hist_2 = get_cliped_dist(e,s,False,test_hist_2,clip=15)
                                    test_feat_curr.append(test_hist_1)
                                    test_feat_curr.append(test_hist_2)

                                    hist_1 = just_spd_lm_file[mbid][s10,e10,:,0]
                                    hist_2 = just_spd_lm_file[mbid][e10,s10,:,1]
                                    hist_1 = get_cliped_dist(s,e,True,hist_1,clip=15)
                                    hist_2 = get_cliped_dist(e,s,False,hist_2,clip=15)
                                    feat_curr.append(hist_1)
                                    feat_curr.append(hist_2)
                                feat_curr = np.concatenate(feat_curr, axis=-1)
                                test_feat_curr = np.concatenate(test_feat_curr, axis=-1)
                                feat.append(feat_curr)
                                test_feat.append(test_feat_curr)
                        else:
                            feat = []
                            test_feat = []
                            for row in df.iterrows():
                                mbid = row[1]['mbid']
                                test_spd_full = np.array(spd_tempo[mbid])
#                                 test_spd_full = np.array(just_spd_lm_file[mbid])
                                spd_full = np.array(just_spd_lm_file[mbid])
                                test_feat.append(np.reshape(test_spd_full, [-1]))
                                feat.append(np.reshape(spd_full, [-1]))
                        feat = np.array(feat)
                        test_feat = np.array(test_feat)
                        loo = LeaveOneOut()
                        y_pred = np.zeros([n_rows,n_labels])
                        print('wd',wd)
                        if only_save:
                            spd_knn.fit(feat, y_labels)
                            with open('data/RagaDataset/{}/model/spd_knn_{}.pkl'.format(tradition, wd), 'wb') as pkl_f:
                                pickle.dump(spd_knn, pkl_f)
                        else:
                            for train_index, test_index in loo.split(feat):  
                                spd_knn.fit(feat[train_index], y_labels[train_index])
                                y_pred_proba = spd_knn.predict(test_feat[test_index])
                                y_pred[test_index[0]] = y_pred_proba[0]
                            file_pred += weights[int(wd/10)]*y_pred
                    file_pred = np.argmax(file_pred, axis=1)
                    pred_df = pd.DataFrame()
                    pred_df['pred'] = file_pred
                    pred_df['labels'] = y_labels
                    pred_df.to_csv(f'Tempo_pred_{t}.csv', index=False)
                    accuracy = np.mean(file_pred==y_labels)*100
                    print(f'{tradition}_{t}_{accuracy}')

   

In [89]:
for t in np.linspace(0.25,2, 8):
    print(t)
    train_tempo_model('Hindustani', t)

0.25
a7f6551d-9696-4713-becb-b22acf71a7db


KeyboardInterrupt: 

In [105]:
t = 0.75
train_tempo_model('Hindustani', t)

a7f6551d-9696-4713-becb-b22acf71a7db
d0781259-2c63-4b35-8ca3-75c19a98e275
99dcaebe-ab49-4fa3-ab6c-a9458143af8e
86d7779a-1610-4514-83a6-883997250abe
c02f714f-e4e6-4da6-af07-cea8a03c0e5c
3f66f0dc-73b7-48c4-8ad7-1f1a7dacd8bb
baea9093-d2c8-4529-8bc8-02ce7554c555
566d6c64-ae66-45e7-9f91-e33cd6c0f18f
7a383e3f-b0a6-4aca-92df-e53c3d928a2d
59ebf93e-fb88-4ac1-b400-000ffbc7ae3d
a0052c33-34b7-4e68-9dcd-121410c6ce36
89224046-f086-4530-b2a3-70f9ae697faf
7084ac88-542c-4e84-8eda-dc9735201d18
a5b15aae-c911-4df5-944d-1d0d95b8b5b0
31bc8850-a7cb-4a9f-bb4f-863e0e88728f
c35ea067-d200-40b6-ad61-241b206c97ed
6c0c6e87-c78c-44df-8955-3595575a06a7
1174dd02-87d2-4be3-bae1-d222b8a5f5a4
e516312d-00c4-4ccc-8875-9c84f0f30207
57db6960-6038-4b48-b22f-a10d22752767
98e14bae-50ae-4c9b-bb39-693ef89fc5b2
8d01b397-5d3f-4fdd-8c9a-a8a3a4e18709
01df0bee-b090-4b1d-bbdc-84e1c436635b
f5a6ba75-7be0-4a67-b1c3-3e49b060d384
6e4dea78-ec22-479a-9e3c-c253a6deba7c
1f5947f8-e4b8-47e4-86a6-8a817dbf8554
b0434093-2d8b-4c8d-aaa8-7fb44bd1ce0e
7

27994cb8-23ae-4c32-959e-625b51727412
28dd1d68-ee23-47f4-8776-33cc18adc44a
0efd0dde-018c-4c30-972b-9dc14c6f09b5
50a45482-2a3e-44a8-beea-1320f529fa00
5e9d1ce8-9dcf-4c3c-a58a-7bc8734791d0
73f6dda2-50bf-416f-8b77-2de40d24b455
2f76f164-f02c-4b22-8bc0-4fe5500039b1
56c945ef-f5ed-4b03-ace1-80ed9b7acf25
8d2eb825-77d6-4f61-ae4c-1a8639dd912b
b63f8746-775e-447b-8bd5-1f32c4ef0b44
a01ac873-dff4-420f-9fd8-490dd87c58e9
cc7999c1-feb7-4fa7-8b45-7e6fc39c0ec6
e40375a2-01d2-4137-823a-3e62503baf98
41e85340-5071-4cdf-a988-6e0aabed3dd6
d29c44a0-d688-47f1-8d83-8c07f0b76b22
15fae9e1-01a0-4b4b-bcc9-4c84321a66a0
7060e016-7772-45c9-bfc6-8d3911ae0a06
409903ce-cfe7-41fd-b42b-3b7790c0e917
a655cf94-bb5f-4971-897a-9c20cdab828b
e23eb921-de6e-452b-bc2f-7fbd0772c4fd
d7510269-b26c-4735-a491-245f3c732a58
3124479b-5118-4cf3-823f-8fefad45e586
62ccf764-ddbd-49d4-a05a-cda03f4192a6
b20456b8-7abc-4158-bb53-5ea8d80e5999
182dd9f9-d3b7-4ebc-9d18-9e0544c6e8cc
7d3f5b32-34f2-402c-b3b3-a87f1dedd3a8
cdd148c7-07e4-46c9-88d9-fbbfcea2edf5
c

In [106]:
t = 0.5
train_tempo_model('Hindustani', t)

a7f6551d-9696-4713-becb-b22acf71a7db
d0781259-2c63-4b35-8ca3-75c19a98e275
99dcaebe-ab49-4fa3-ab6c-a9458143af8e
86d7779a-1610-4514-83a6-883997250abe
c02f714f-e4e6-4da6-af07-cea8a03c0e5c
3f66f0dc-73b7-48c4-8ad7-1f1a7dacd8bb
baea9093-d2c8-4529-8bc8-02ce7554c555
566d6c64-ae66-45e7-9f91-e33cd6c0f18f
7a383e3f-b0a6-4aca-92df-e53c3d928a2d
59ebf93e-fb88-4ac1-b400-000ffbc7ae3d
a0052c33-34b7-4e68-9dcd-121410c6ce36
89224046-f086-4530-b2a3-70f9ae697faf
7084ac88-542c-4e84-8eda-dc9735201d18
a5b15aae-c911-4df5-944d-1d0d95b8b5b0
31bc8850-a7cb-4a9f-bb4f-863e0e88728f
c35ea067-d200-40b6-ad61-241b206c97ed
6c0c6e87-c78c-44df-8955-3595575a06a7
1174dd02-87d2-4be3-bae1-d222b8a5f5a4
e516312d-00c4-4ccc-8875-9c84f0f30207
57db6960-6038-4b48-b22f-a10d22752767
98e14bae-50ae-4c9b-bb39-693ef89fc5b2
8d01b397-5d3f-4fdd-8c9a-a8a3a4e18709
01df0bee-b090-4b1d-bbdc-84e1c436635b
f5a6ba75-7be0-4a67-b1c3-3e49b060d384
6e4dea78-ec22-479a-9e3c-c253a6deba7c
1f5947f8-e4b8-47e4-86a6-8a817dbf8554
b0434093-2d8b-4c8d-aaa8-7fb44bd1ce0e
7

27994cb8-23ae-4c32-959e-625b51727412
28dd1d68-ee23-47f4-8776-33cc18adc44a
0efd0dde-018c-4c30-972b-9dc14c6f09b5
50a45482-2a3e-44a8-beea-1320f529fa00
5e9d1ce8-9dcf-4c3c-a58a-7bc8734791d0
73f6dda2-50bf-416f-8b77-2de40d24b455
2f76f164-f02c-4b22-8bc0-4fe5500039b1
56c945ef-f5ed-4b03-ace1-80ed9b7acf25
8d2eb825-77d6-4f61-ae4c-1a8639dd912b
b63f8746-775e-447b-8bd5-1f32c4ef0b44
a01ac873-dff4-420f-9fd8-490dd87c58e9
cc7999c1-feb7-4fa7-8b45-7e6fc39c0ec6
e40375a2-01d2-4137-823a-3e62503baf98
41e85340-5071-4cdf-a988-6e0aabed3dd6
d29c44a0-d688-47f1-8d83-8c07f0b76b22
15fae9e1-01a0-4b4b-bcc9-4c84321a66a0
7060e016-7772-45c9-bfc6-8d3911ae0a06
409903ce-cfe7-41fd-b42b-3b7790c0e917
a655cf94-bb5f-4971-897a-9c20cdab828b
e23eb921-de6e-452b-bc2f-7fbd0772c4fd
d7510269-b26c-4735-a491-245f3c732a58
3124479b-5118-4cf3-823f-8fefad45e586
62ccf764-ddbd-49d4-a05a-cda03f4192a6
b20456b8-7abc-4158-bb53-5ea8d80e5999
182dd9f9-d3b7-4ebc-9d18-9e0544c6e8cc
7d3f5b32-34f2-402c-b3b3-a87f1dedd3a8
cdd148c7-07e4-46c9-88d9-fbbfcea2edf5
c